# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3640059af0>
├── 'a' --> tensor([[ 0.0921,  0.4500, -0.0551],
│                   [-0.7410,  1.9680, -0.6537]])
└── 'x' --> <FastTreeValue 0x7f3640059940>
    └── 'c' --> tensor([[-0.0095,  1.4761, -1.0964, -0.5983],
                        [-0.6769,  1.7113,  0.5884,  0.0904],
                        [ 2.3243,  1.0271,  0.3874,  0.8383]])

In [4]:
t.a

tensor([[ 0.0921,  0.4500, -0.0551],
        [-0.7410,  1.9680, -0.6537]])

In [5]:
%timeit t.a

58 ns ± 0.0554 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3640059af0>
├── 'a' --> tensor([[ 2.0797, -0.2619,  0.4085],
│                   [-0.4121, -2.1026, -1.8509]])
└── 'x' --> <FastTreeValue 0x7f3640059940>
    └── 'c' --> tensor([[-0.0095,  1.4761, -1.0964, -0.5983],
                        [-0.6769,  1.7113,  0.5884,  0.0904],
                        [ 2.3243,  1.0271,  0.3874,  0.8383]])

In [7]:
%timeit t.a = new_value

58.9 ns ± 0.0435 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0921,  0.4500, -0.0551],
               [-0.7410,  1.9680, -0.6537]]),
    x: Batch(
           c: tensor([[-0.0095,  1.4761, -1.0964, -0.5983],
                      [-0.6769,  1.7113,  0.5884,  0.0904],
                      [ 2.3243,  1.0271,  0.3874,  0.8383]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0921,  0.4500, -0.0551],
        [-0.7410,  1.9680, -0.6537]])

In [11]:
%timeit b.a

59.7 ns ± 0.0358 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4319,  0.1050, -0.4790],
               [ 0.4189,  1.3031, -0.9238]]),
    x: Batch(
           c: tensor([[-0.0095,  1.4761, -1.0964, -0.5983],
                      [-0.6769,  1.7113,  0.5884,  0.0904],
                      [ 2.3243,  1.0271,  0.3874,  0.8383]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.0705 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

840 ns ± 5.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 29.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 809 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 321 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3589347fa0>
├── 'a' --> tensor([[[ 0.0921,  0.4500, -0.0551],
│                    [-0.7410,  1.9680, -0.6537]],
│           
│                   [[ 0.0921,  0.4500, -0.0551],
│                    [-0.7410,  1.9680, -0.6537]],
│           
│                   [[ 0.0921,  0.4500, -0.0551],
│                    [-0.7410,  1.9680, -0.6537]],
│           
│                   [[ 0.0921,  0.4500, -0.0551],
│                    [-0.7410,  1.9680, -0.6537]],
│           
│                   [[ 0.0921,  0.4500, -0.0551],
│                    [-0.7410,  1.9680, -0.6537]],
│           
│                   [[ 0.0921,  0.4500, -0.0551],
│                    [-0.7410,  1.9680, -0.6537]],
│           
│                   [[ 0.0921,  0.4500, -0.0551],
│                    [-0.7410,  1.9680, -0.6537]],
│           
│                   [[ 0.0921,  0.4500, -0.0551],
│                    [-0.7410,  1.9680, -0.6537]]])
└── 'x' --> <FastTreeValue 0x7f35893470a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 73.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f36400cd070>
├── 'a' --> tensor([[ 0.0921,  0.4500, -0.0551],
│                   [-0.7410,  1.9680, -0.6537],
│                   [ 0.0921,  0.4500, -0.0551],
│                   [-0.7410,  1.9680, -0.6537],
│                   [ 0.0921,  0.4500, -0.0551],
│                   [-0.7410,  1.9680, -0.6537],
│                   [ 0.0921,  0.4500, -0.0551],
│                   [-0.7410,  1.9680, -0.6537],
│                   [ 0.0921,  0.4500, -0.0551],
│                   [-0.7410,  1.9680, -0.6537],
│                   [ 0.0921,  0.4500, -0.0551],
│                   [-0.7410,  1.9680, -0.6537],
│                   [ 0.0921,  0.4500, -0.0551],
│                   [-0.7410,  1.9680, -0.6537],
│                   [ 0.0921,  0.4500, -0.0551],
│                   [-0.7410,  1.9680, -0.6537]])
└── 'x' --> <FastTreeValue 0x7f36400cd430>
    └── 'c' --> tensor([[-0.0095,  1.4761, -1.0964, -0.5983],
                        [-0.6769,  1.7113,  0.5884,  0.0904],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 94.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.9 µs ± 54.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0921,  0.4500, -0.0551],
                [-0.7410,  1.9680, -0.6537]],
       
               [[ 0.0921,  0.4500, -0.0551],
                [-0.7410,  1.9680, -0.6537]],
       
               [[ 0.0921,  0.4500, -0.0551],
                [-0.7410,  1.9680, -0.6537]],
       
               [[ 0.0921,  0.4500, -0.0551],
                [-0.7410,  1.9680, -0.6537]],
       
               [[ 0.0921,  0.4500, -0.0551],
                [-0.7410,  1.9680, -0.6537]],
       
               [[ 0.0921,  0.4500, -0.0551],
                [-0.7410,  1.9680, -0.6537]],
       
               [[ 0.0921,  0.4500, -0.0551],
                [-0.7410,  1.9680, -0.6537]],
       
               [[ 0.0921,  0.4500, -0.0551],
                [-0.7410,  1.9680, -0.6537]]]),
    x: Batch(
           c: tensor([[[-0.0095,  1.4761, -1.0964, -0.5983],
                       [-0.6769,  1.7113,  0.5884,  0.0904],
                       [ 2.3243,  1.0271,  0.3874,  0.8383]],
         

In [26]:
%timeit Batch.stack(batches)

77.3 µs ± 168 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0921,  0.4500, -0.0551],
               [-0.7410,  1.9680, -0.6537],
               [ 0.0921,  0.4500, -0.0551],
               [-0.7410,  1.9680, -0.6537],
               [ 0.0921,  0.4500, -0.0551],
               [-0.7410,  1.9680, -0.6537],
               [ 0.0921,  0.4500, -0.0551],
               [-0.7410,  1.9680, -0.6537],
               [ 0.0921,  0.4500, -0.0551],
               [-0.7410,  1.9680, -0.6537],
               [ 0.0921,  0.4500, -0.0551],
               [-0.7410,  1.9680, -0.6537],
               [ 0.0921,  0.4500, -0.0551],
               [-0.7410,  1.9680, -0.6537],
               [ 0.0921,  0.4500, -0.0551],
               [-0.7410,  1.9680, -0.6537]]),
    x: Batch(
           c: tensor([[-0.0095,  1.4761, -1.0964, -0.5983],
                      [-0.6769,  1.7113,  0.5884,  0.0904],
                      [ 2.3243,  1.0271,  0.3874,  0.8383],
                      [-0.0095,  1.4761, -1.0964, -0.5983],
                      [-0.6769,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 246 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
